In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("241212_01_MLlib_regression").getOrCreate()

24/12/12 15:51:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# data load

In [7]:
train_df = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/house_train.csv')

In [4]:
test_df = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/house_test.csv')

In [5]:
# sale price : label
# features : ???

In [8]:
train_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

# 전처리
- 결측치 = 0
- 숫자 타입 통일 (이미 되어있으므로 skip)

In [9]:
# 결측치 처리
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

# 인코딩
* 문자형 > 숫자형 (1, 2, 3, 4)

In [12]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder

In [13]:
string_columns = ['Neighborhood']
# 1,2,3,4 로 값을 단순화
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index")   for col in string_columns]

In [15]:
# onehotencoding : 범주형변수 1,2,3,4, -> 1로 바꾸는 인코딩 (모델에 미치는 영향을 동일하게 세팅)
# 모두 1로 바꾼다.
encoders = [OneHotEncoder(inputCol=col+"_index", outputCol=col+"_encoded")   for col in string_columns]

# features selection

In [19]:
numeric_columns = ["LotArea", "OverallQual", "OverallCond", "YearBuilt", "YearRemodAdd", 
    "1stFlrSF", "2ndFlrSF", "GrLivArea", "GarageCars", "GarageArea"]

In [20]:
assembler_inputs = [col+"_encoded" for col in string_columns] + numeric_columns

# assembler

In [21]:
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

# label selection

In [22]:
train_df = train_df.withColumnRenamed("SalePrice", "label")

# pipline 설정

In [23]:
from pyspark.ml import Pipeline

In [24]:
pipeline = Pipeline(stages = indexers+encoders+[assembler])

In [25]:
# pipeline 실행
pipeline_model = pipeline.fit( train_df )

In [26]:
train_transfmodel = pipeline_model.transform( train_df )

In [27]:
train_transformed = pipeline_model.transform( train_df )

# 예측 > 회귀  모델 학습 > 평가 > 예측

In [29]:
from pyspark.ml.regression import LinearRegression

In [30]:
lr = LinearRegression(featuresCol="features", labelCol="label")
lr_model = lr.fit(train_transformed)

24/12/12 16:48:15 WARN Instrumentation: [76dc05f6] regParam is zero, which might cause numerical instability and overfitting.
24/12/12 16:48:16 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/12/12 16:48:16 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/12/12 16:48:16 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/12/12 16:48:16 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [ ]:
# 평가 데이터 를 이용한 평가> FIT - 모델 맞춤 과정, 테스트 데이터에 의해 규칙이 변화

test_transformed = pipeline_model.transform( test_df )
predictions = lr_model.transform(test_transformed)